In [1]:
import numpy as np
import matplotlib.pyplot as plt
import handytools.navigator as nav
import spectools.models.models as mdl

preprocess = 2

In [2]:
model = mdl.get_vgg16()
R_baseline = nav.npload("/src", "results", f"responses_VGG16", f"key=8_hollow=0_scale=1_light=1_lw=1_preproc={preprocess}.npy")
R_scaled = nav.npload("/src", "results", f"responses_VGG16", f"key=8_hollow=0_scale=2_light=1_lw=1_preproc={preprocess}.npy")

In [3]:
params = mdl.get_parameters("vgg16")

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [11]:
params["features.11.weight"].shape

torch.Size([512, 256, 3, 3])

In [8]:
params.keys()

dict_keys(['features.0.weight', 'features.0.bias', 'features.3.weight', 'features.3.bias', 'features.6.weight', 'features.6.bias', 'features.8.weight', 'features.8.bias', 'features.11.weight', 'features.11.bias', 'features.13.weight', 'features.13.bias', 'features.16.weight', 'features.16.bias', 'features.18.weight', 'features.18.bias', 'classifier.0.weight', 'classifier.0.bias', 'classifier.3.weight', 'classifier.3.bias', 'classifier.6.weight', 'classifier.6.bias'])